## This notebook is just to take in xls/xlsx files and write out csv's for normal use

In [57]:
def xuni(s):
    # handle numerics
    if type(s) == float:
        return s
    elif type(s) == int:
        return s
    elif not s:
        return u''
    elif type(s) == unicode:
        return s.encode('utf-8')
    elif type(s) == str:
        return unicode(s, 'utf-8')
    else: 
        return s
    
def xstr(s):
    # handle numerics
    if type(s) == float:
        return s
    elif type(s) == int:
        return s
    elif not s:
        return ''
    elif type(s) == unicode:
        return str(s)
    elif type(s) == str:
        return s
    else: 
        return s
    

In [48]:
import csv
import openpyxl
import xlrd

def write_excel_to_csv(filename, sheet_index=0, strip_newline=True, unicode=True, outname=None):
    """If you want to write out a different sheet than the first, input the sheet index (from 0)
        outname is the file name WITHOUT .csv"""
    split = filename.split('.')
    ext = split[-1]
    if ext == 'xlsx':
        if outname:
            fname = outname
        else:
            fname = '.'.join(split[:-1])
        wb = openpyxl.load_workbook(filename)
        sh = wb.worksheets[sheet_index]
        with open(fname + '.csv', 'wb') as f:
            c = csv.writer(f)
            for r in sh.rows:
                if strip_newline:
                    if unicode:
                        line = [xuni(cell.value).strip('\n').strip('\r') for cell in r]
                    else:
                        line = [xstr(cell.value).strip('\n').strip('\r') for cell in r]
                else:
                    if unicode:
                        line = [xuni(cell.value) for cell in r]
                    else:
                        line = [xstr(cell.value) for cell in r]
                c.writerow(line)
        return fname+'.csv'
    elif ext == 'xls':
        if outname:
            fname = outname
        else:
            fname = '.'.join(split[:-1])
        wb = xlrd.open_workbook(filename)
        sh = wb.sheet_by_index(sheet_index)
        with open(fname + '.csv', 'wb') as f:
            c = csv.writer(f)
            num_cols = sh.ncols   # Number of columns
            for row_idx in range(0, sh.nrows):
                if strip_newline:
                    if unicode:
                        line = [xuni(sh.cell(row_idx, col_idx).value).strip('\n').strip('\r') for col_idx in range(0, num_cols)]
                    else:
                        line = [xstr(sh.cell(row_idx, col_idx).value).strip('\n').strip('\r') for col_idx in range(0, num_cols)]
                else:
                    if unicode:
                        line = [xuni(sh.cell(row_idx, col_idx).value) for col_idx in range(0, num_cols)]
                    else:
                        line = [xstr(sh.cell(row_idx, col_idx).value) for col_idx in range(0, num_cols)]
                c.writerow(line)
        return fname+'.csv'

In [55]:
def csv_col_names(csvfile):
    with open(csvfile, 'rU') as f:
        reader = csv.reader(f)
        headers = reader.next()    
    return headers

def write_select_columns_to_csv(infile, outfile, colnames):
    """ Takes a csv file, outputs a csv file with cols columns 
        NOTE: ASSUMES THE FIRST LINE IS ALWAYS THE HEADER"""
    if infile.split('.')[-1] != 'csv':
        print "INPUT NEEDS TO BE CSV. EXITING"
        return
    with open(infile, 'rU') as f:
        reader = csv.reader(f)
        with open(outfile, 'w') as w:
            writer = csv.writer(w)
            col_is = []
            for i, row in enumerate(reader):
                if i == 0: # headers
                    col_is = [ row.index(coln) for coln in colnames]
                    writer.writerow(colnames)
                    continue
                # rest of the data
                newrow = [ row[col_i] for col_i in col_is ]
                writer.writerow(newrow)
    print "DONE"
    

In [50]:
import os
os.listdir('.')

['.DS_Store',
 '.ipynb_checkpoints',
 'CSV Writer.ipynb',
 'Doc Embeddings Presi.ipynb',
 'Evaluation Sample.ipynb',
 'Foodborne Twitter Tests.ipynb',
 'kdd2011-pldp-final.pdf',
 'Labeled Tweets Analysis.ipynb',
 'LDA.ipynb',
 'ml_example.zip',
 'pull_yelp.xlsx',
 'pull_yelp_raw.csv',
 'PyHealth test.ipynb',
 'reviews_foodborne.csv',
 'stream_incidents1.json',
 'stream_tweets1.json',
 'Twitter sandbox.ipynb',
 'Twitter Stream.ipynb',
 'Yelp Reviews_dataset for paper data for Fotis.xlsx',
 'Yelp Sandbox.ipynb',
 'yelp_annotated_analysis.csv',
 'yelp_annotated_analysis.xls',
 'yelp_mult_data.xlsx']

In [58]:
og_file = 'yelp_annotated_analysis.xls'
incsv = write_excel_to_csv(og_file, sheet_index=6, strip_newline = False)
print incsv
cols = csv_col_names(incsv)
print "HEADERS: %r" % cols

yelp_annotated_analysis.csv
HEADERS: ['ID', 'Date_Review', 'URL', 'User', 'Business', 'Review', 'Keyword', 'Sick', 'Sick_2', 'Multiple', 'Incubation', 'Interest', 'Score', 'Foodborne_Disease', 'Occurred_Less_Than_4_Weeks', '2_Or_More_Ill', 'Severe_Illness_Occur', 'Action_Item', '311_Report', 'FoodborneAsBoolean', 'NumIsSick']


In [59]:
mycols = ['Review', 'FoodborneAsBoolean']
outfile = "reviews_foodborne.csv"
write_select_columns_to_csv(incsv, outfile, mycols)

DONE
